In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import math
from ordered_set import OrderedSet
np.seterr(divide = 'ignore') 
np.seterr(over="ignore")
np.seterr(all="ignore")
np.seterr(under="ignore")
np.seterr(invalid="ignore")


{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [3]:
cd ..

/Users/helensiavelis/Desktop/Caltech/Senior/Spring 2023/CS 145/CS-145


In [4]:
cd MarketMaven/data

/Users/helensiavelis/Desktop/Caltech/Senior/Spring 2023/CS 145/CS-145/MarketMaven/data


In [28]:
data

,permno,date,returns
1,43123,2010-01-29,-0.087559
4,87532,2010-01-29,-0.002469
8,83534,2010-01-29,-0.136029
10,79839,2010-01-29,-0.089093
11,43350,2010-01-29,-0.005048
...,...,...,...
463121,19937,2020-12-31,0.341228
463122,12570,2020-12-31,0.062770
463123,13188,2020-12-31,0.069342
463124,18202,2020-12-31,-0.044745


In [26]:
data = pd.read_csv('monthly_stock.csv')
remove_indexes = data[data['returns'] == 0].index
data.drop(remove_indexes, inplace=True)
data = data.drop("monthly_transaction_id", axis=1)
data = data.drop("price", axis=1)
data = data.drop("shares_outstanding", axis=1)
data = data.drop("share_type", axis=1)
data = data.drop("exchange", axis=1)

In [27]:
## drop out any stocks that are not currently in the database
# find the last index of each permno. if not in 2020-12-31, drop it 
for permno in set(data['permno'].values):
    last_index = data[data['permno'] == permno].tail(1)
    if last_index['date'].item() < '2020-12-31':
        remove_indexes = data[data['permno'] == permno].index
        data.drop(remove_indexes, inplace=True)
    


In [29]:
def create_correlation_matrix(permnos):
    lst = []
    counter = 0 
    for i in permnos:
        sub_lst = []
        permno_i_data = data[data['permno']== i]
        for j in permnos:
            permno_j_data = data[data['permno']== j]
            
            intersection = list(OrderedSet(list(permno_i_data['date'].values)) & OrderedSet(list(permno_j_data['date'].values)))
            returns_i = list(permno_i_data[permno_i_data['date'].isin(intersection)]['returns'].values)
            returns_j = list(permno_j_data[permno_j_data['date'].isin(intersection)]['returns'].values)
            sub_lst.append(np.corrcoef(returns_i, returns_j)[1, 0])
        
        lst.append(sub_lst)
        counter +=1
        if counter % 10 == 0:
            print(counter)
    return lst 

In [30]:
def adj_matrix(correlation_matrix, theta):
    lst = []
    for row_index in range(len(correlation_matrix)):
        sub_lst = []
        for col_index in range(len(correlation_matrix[row_index])):
            if row_index != col_index and abs(correlation_matrix[row_index][col_index]) > theta:
                sub_lst.append(1)
            else:
                sub_lst.append(0)
        lst.append(sub_lst)
    return lst


In [10]:
## pass in permnos to creat_correlation_matrix
## then pass in correlation matrix to theta 
## write list to csv 

In [11]:
# now build the correlation network 

In [31]:
len(data['permno'].unique())

3537

In [ ]:
correlation_matrix = create_correlation_matrix(sorted(list(data['permno'].unique())))

/Users/helensiavelis/miniforge3/lib/python3.9/site-packages/numpy/lib/function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260


In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(correlation_matrix, cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
np.mean(correlation_matrix)

In [ ]:
theta = .9
def adj_matrix(correlation_matrix):
    lst = []
    for row_index in range(len(correlation_matrix)):
        sub_lst = []
        for col_index in range(len(correlation_matrix[row_index])):
            if row_index != col_index and abs(correlation_matrix[row_index][col_index]) > theta:
                sub_lst.append(1)
            else:
                sub_lst.append(0)
        lst.append(sub_lst)
    return lst

In [ ]:
def create_correlation_network():
    g = nx.Graph()
    for key in stock_dict.keys():
        g.add_node(key)
        
    adj_list = adj_matrix(create_correlation_matrix())
    dict_keys = list(stock_dict.keys())
    
    for row_index in range(len(adj_list)):
        for col_index in range(len(adj_list[row_index])):
            if adj_list[row_index][col_index]:
                g.add_edge(dict_keys[row_index], dict_keys[col_index])
    return g
    

In [ ]:
g = create_correlation_network()

plt.figure(figsize=(20, 20))
nx.draw_spring(g)

#### NOW FIND THE CENTRALITY MEASURES

The general ideas is to select the stocks with high centralities, which measure different importances of the stock 
relative to the market.

IN THE FUTURE USE MEASURES TO MAKE A HEDGE, THI strategy is fine for now since our data is from a time period 
of rapid market growth, so the choosing stocks that are highly correlated with the market is fine. In the future, 
we need to create a portfolio that hedges this risk AND expand our time period to the 2020s. 


In [ ]:
def solve_optimization_problem():
    return 1/3  

In [ ]:
# Find the average centralities of all stocks
def find_average_centralities(graph):
    average_centrailites = dict()
    scale_factor = solve_optimization_problem()
    
    degree_centrality = nx.degree_centrality(graph)
    closeness_centrality = nx.closeness_centrality(graph)
    betweenness_centrality = nx.betweenness_centrality(graph)
    
    for node in graph.nodes():
        average_centrailites[node] = scale_factor * (degree_centrality[node] + closeness_centrality[node] + betweenness_centrality[node])
        
        
    return average_centrailites

In [ ]:
def get_top_stocks(top_percent, graph):
    c_avgs = find_average_centralities(graph)
    sorted_c_avgs = sorted(c_avgs, key=c_avgs.get, reverse=True)
    num_stocks = math.floor(top_percent * len(c_avgs.keys()))
    return sorted_c_avgs[:num_stocks]

def get_bottom_stocks(bottom_percent, graph):
    c_avgs = find_average_centralities(graph)
    sorted_c_avgs = sorted(c_avgs, key=c_avgs.get, reverse=False)
    num_stocks = math.floor(bottom_percent * len(c_avgs.keys()))
    return sorted_c_avgs[:num_stocks]


In [ ]:
print(get_top_stocks(.10, g))
print(get_bottom_stocks(0.10, g))

In [ ]:
ffm = pd.read_csv('F-F_Research_Data_Factors_daily.CSV') 

In [ ]:
## parse through 